# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [2]:
import pandas as pd
import time

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [3]:

filename_features = "Dados/features.txt"
filename_labels = "Dados/activity_labels.txt"

filename_subtrain = "Dados/train/subject_train.txt"
filename_xtrain = "Dados/train/X_train.txt"
filename_ytrain = "Dados/train/y_train.txt"

filename_subtest = "Dados/test/subject_test.txt"
ffilename_xtest = "Dados/test/X_test.txt"
filename_ytest = "Dados/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

C:\Users\ricme\AppData\Local\Temp\ipykernel_24032\3738210203.py:12: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
C:\Users\ricme\AppData\Local\Temp\ipykernel_24032\3738210203.py:15: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
C:\Users\ricme\AppData\Local\Temp\ipykernel_24032\3738210203.py:19: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [4]:
%%time

# Inicializar o classificador de árvore de decisão com ccp_alpha=0.001
dt_classifier = DecisionTreeClassifier(ccp_alpha=0.001)

dt_classifier.fit(X_train, y_train.values.ravel())

# Previsões na base de treino
train_predictions = dt_classifier.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

# Previsões na base de teste
test_predictions = dt_classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

train_accuracy, test_accuracy

CPU times: total: 3.38 s
Wall time: 4.62 s


(0.9757889009793254, 0.8812351543942993)

- Tivemos uma boa acurácia na base de treino, mas uma acurácia mais baixa na base de teste. Isso já era esperado.
- O tempo de processamento foi de aproximadamente 3 segundosb.
- Verificar a possibilidade do modelo está sobfrendo overfitting.

## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [5]:
%%time
# Aplicar PCA para extrair apenas a primeira componente principal
pca_1 = PCA(n_components=1)

x_train_pca_1 = pca_1.fit_transform(X_train)  # Aplicando PCA ao conjunto de treino 
x_test_pca_1 = pca_1.transform(X_test)        # Aplicando PCA ao conjunto de teste 

# Treinar a árvore de decisão com a primeira componente principal
dt_classifier_pca = DecisionTreeClassifier(ccp_alpha=0.001)


dt_classifier_pca.fit(x_train_pca_1, y_train.values.ravel())

# Previsões na base de treino
train_predictions_pca = dt_classifier_pca.predict(x_train_pca_1)
train_accuracy_pca = accuracy_score(y_train, train_predictions_pca)

# Previsões na base de teste
test_predictions_pca = dt_classifier_pca.predict(x_test_pca_1)
test_accuracy_pca = accuracy_score(y_test, test_predictions_pca)

train_accuracy_pca, test_accuracy_pca

CPU times: total: 875 ms
Wall time: 398 ms


(0.499727965179543, 0.45707499151679676)

- O tempo de processamento foi de aproximadamente 0.5 segundos, bem menor que o tempo da árvore sem PCA.
- Porém, como utilizamos somente 1 componente principal, a acurácia foi bem menor que a árvore sem PCA.


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [6]:
%%time
# Números de componentes a serem testados. Acrescentei mais um número de componentes para testar.
component_numbers = [1, 2, 5, 10, 50, 100]

# Inicializar listas para armazenar os resultados
results = []

# Loop para testar cada número de componentes
for n_components in component_numbers:
    # Aplicar PCA
    pca_n = PCA(n_components=n_components)
    x_train_pca_n = pca_n.fit_transform(X_train)
    x_test_pca_n = pca_n.transform(X_test)

    # Treinar a árvore de decisão
    dt_classifier_pca_n = DecisionTreeClassifier(ccp_alpha=0.001)
    start_time_pca_n = time.time()
    dt_classifier_pca_n.fit(x_train_pca_n, y_train.values.ravel())
    training_time_pca_n = time.time() - start_time_pca_n

    # Avaliar acurácia
    train_accuracy_pca_n = accuracy_score(y_train, dt_classifier_pca_n.predict(x_train_pca_n))
    test_accuracy_pca_n = accuracy_score(y_test, dt_classifier_pca_n.predict(x_test_pca_n))

    # Adicionar resultados à lista
    results.append({
        "n_components": n_components,
        "training_time": training_time_pca_n,
        "train_accuracy": train_accuracy_pca_n,
        "test_accuracy": test_accuracy_pca_n
    })

# Converter resultados para um DataFrame para melhor visualização
results_df = pd.DataFrame(results)
results_df

CPU times: total: 5.7 s
Wall time: 3.4 s


,n_components,training_time,train_accuracy,test_accuracy
0,1,0.045986,0.499728,0.457075
1,2,0.040001,0.612758,0.584662
2,5,0.048996,0.846028,0.788938
3,10,0.099998,0.891866,0.814388
4,50,0.533997,0.918118,0.830675
5,100,1.014000,0.927911,0.827621


- O tempo de processamento foi aumentando conforme aumentamos o número de componentes.
- A acurácia foi aumentando conforme aumentamos o número de componentes.
- A acurácia na base de treino foi aumentando conforme aumentamos o número de componentes, mas a acurácia na base de teste foi aumentando significativamente até 10 componentes e depois estabilizou.

## Conclua

- O que aconteceu com a acurácia?
  - Foi gradativamente aumentando conforme aumentamos o número de componentes

- O que aconteceu com o tempo de processamento?
    - Também foi gradativamente aumentando conforme aumentamos o número de componentes

### BÔNUS: Teste o número de componentes com validação cruzada

In [7]:
%%time

# Inicializar lista para armazenar os resultados da validação cruzada
cv_results = []
component_numbers = [1, 2, 5, 10, 50, 100]


# Loop para aplicar validação cruzada com diferentes números de componentes principais
for n_components in component_numbers:
    # Aplicar PCA
    pca_n_cv = PCA(n_components=n_components)
    x_train_pca_n_cv = pca_n_cv.fit_transform(X_train)

    # Inicializar a árvore de decisão
    dt_classifier_pca_cv = DecisionTreeClassifier(ccp_alpha=0.001)

    # Realizar validação cruzada
    cv_scores = cross_val_score(dt_classifier_pca_cv, x_train_pca_n_cv, y_train.values.ravel(), cv=5, scoring='accuracy')
    cv_mean_accuracy = cv_scores.mean()

    # Adicionar resultados à lista
    cv_results.append({
        "n_components": n_components,
        "cv_mean_accuracy": cv_mean_accuracy
    })

# Converter resultados para um DataFrame para melhor visualização
cv_results_df = pd.DataFrame(cv_results)
cv_results_df

CPU times: total: 8.62 s
Wall time: 8.27 s


,n_components,cv_mean_accuracy
0,1,0.475111
1,2,0.573726
2,5,0.754089
3,10,0.793123
4,50,0.782244
5,100,0.786870


- A acurácia foi aumentando conforme aumentamos o número de componentes.
- Mas após 10 componentes, a acurácia estabilizou.

### BÔNUS: Teste o número de componentes com grid search

In [8]:
%%time

# Definir o pipeline que combina PCA e árvore de decisão
pipeline = Pipeline([
    ('pca', PCA()),
    ('dt', DecisionTreeClassifier())
])

# Espaço de parâmetros para GridSearchCV
param_grid = {
    'pca__n_components': [1, 2, 5, 10, 50, 100],
    'dt__ccp_alpha': [0.0005, 0.001, 0.01, 0.1]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Realizar Grid Search
grid_search.fit(X_train, y_train.values.ravel())

# Melhores parâmetros e acurácia correspondente
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

CPU times: total: 2.06 s
Wall time: 27 s


({'dt__ccp_alpha': 0.001, 'pca__n_components': 100}, 0.7992383357149795)

- O melhor número de componentes foi 10 e o melhor ccp_alpha foi 0.001.
- Mostrando que o PCA com 10 componentes e a árvore de decisão com ccp_alpha=0.001 é o melhor modelo para esse caso.